In [1]:
import pandas as pd 
import os

In [44]:
# make dirs 

xnat_root = '/nfs/masi/MCL/xnat'
maintain_root = '/nfs/masi/MCL/QA/maintain_list'
xnat_list = os.listdir(xnat_root)
for xnat_name in xnat_list:
    xname = xnat_name.split('_')[0]
    if xname[:4] == 'xnat' and len(xname[4:]) == 8 and xname[4:] > '20200120':
        if not os.path.exists(maintain_root + '/' + xname[4:]):
            os.mkdir(maintain_root + '/' + xname[4:])
        print (xname)
    
    #print (xname)
print ()

In [67]:
# generate QA_good from QA_bad
data_root = '/nfs/masi/MCL/xnat/xnat20200909'
df = pd.read_csv(data_root + '/dicomQA.csv')
all_list = df['sess'].tolist() 

bad_list = open(data_root + '/QA_bad.txt').readlines()
bad_list = [i.strip() for i in bad_list]
print ("this should be None", set(bad_list) - set(all_list)) 

good_list = set(all_list) - set(bad_list)
f = open(data_root + '/QA_good.txt', 'w')
for item in good_list:
    f.write(item + '\n')
f.close()

this should be None {''}


In [85]:
# the past_root and current_root should be near

past_root = '/nfs/masi/MCL/QA/maintain_list/20200902'
current_root = '/nfs/masi/MCL/QA/maintain_list/20200909'

past_bad = pd.read_csv(past_root + '/uptonow_bad.csv')['sess'].tolist()

past_good = pd.read_csv(past_root + '/uptonow_good.csv')['sess'].tolist()

current_bad = open(current_root + '/QA_bad.txt').readlines()
current_bad = [i.strip().split(' ')[0] for i in current_bad]

current_good = open(current_root + '/QA_good.txt').readlines()
current_good = [i.strip().split(' ')[0] for i in current_good]

total_bad = set(current_bad) | (set(past_bad) - set(current_good))

total_good = set(current_good) | (set(past_good) - set(current_bad))

data = pd.DataFrame()

data['sess'] = list(total_good)

data.to_csv(current_root + '/uptonow_good.csv', index = False)

data = pd.DataFrame()

data['sess'] = list(total_bad)

data.to_csv(current_root + '/uptonow_bad.csv', index = False)

print (len(past_bad), len(past_good), len(total_bad), len(total_good))


331 4733 331 4741


In [21]:
data_list = os.listdir('/nfs/masi/MCL/QA/maintain_list')
print (data_list)
print (sorted(data_list))

['20200815', 'before20191218', '20200901', 'code', '20200206_xinmeng', '20200130', 'before201906', '20200201', '20200306', '20200106', '20200816', '20200504', '20200902', '20200115', '20200311', '20200127', '20200120', '20200522', '20200131', '20200128', '20200512', 'readme.txt', '20200326', '20200110', '20200117', '20200207', '20200222', '20200121', '20200424', '20200909', '20200505']
['20200106', '20200110', '20200115', '20200117', '20200120', '20200121', '20200127', '20200128', '20200130', '20200131', '20200201', '20200206_xinmeng', '20200207', '20200222', '20200306', '20200311', '20200326', '20200424', '20200504', '20200505', '20200512', '20200522', '20200815', '20200816', '20200901', '20200902', '20200909', 'before201906', 'before20191218', 'code', 'readme.txt']


In [86]:
# temp 

past_root = '/nfs/masi/MCL/QA/maintain_list/20200909'

past_bad = pd.read_csv(past_root + '/uptonow_bad.csv')['sess'].tolist()

past_good = pd.read_csv(past_root + '/uptonow_good.csv')['sess'].tolist()

print (set(past_bad) & set(past_good))



{nan}


In [94]:

data_root = '/nfs/masi/MCL/xnat/MCL'
subj_list = os.listdir(data_root)
total_sess = []
f = open('/nfs/masi/MCL/xnat/xnat20200928_makeupjustin/need_sess_path.txt', 'w')
for subj in subj_list:
    sess_list = os.listdir(data_root + '/' + subj)
    for sess in sess_list:
        if sess in need_sess:
            f.write(data_root + '/' + subj + '/' + sess + '\n')
f.close()


In [92]:
print (len((set(total_sess)) & set(past_bad)))

241


In [93]:
need_sess = (set(total_sess)) & set(past_bad)
f = open('/nfs/masi/MCL/xnat/xnat20200928_makeupjustin/need_sess.csv', 'w')
for sess in need_sess:
    f.write(sess + '\n')
f.close()


In [103]:
df = pd.read_csv('/nfs/masi/MCL/xnat/xnat20201002_mightallbad/Riqiang_10_2_2020_16_37_59.csv')

all_sess = df['XNAT_CTSESSIONDATA ID'].tolist()

df1 = pd.read_csv('/nfs/masi/MCL/xnat/xnat20200928_makeupjustin/dicomQA_step1.csv')
df2 = pd.read_csv('/nfs/masi/MCL/xnat/xnat20200928_makeupjustin/dicomQA_step2.csv')

sess0928 = df1['sess'].tolist() + df2['sess'].tolist()

df_pre = pd.read_csv('/nfs/masi/MCL/QA/maintain_list/20200909/uptonow_good.csv')
sess_pre = df_pre['sess'].tolist()

In [104]:
need = set(all_sess) - set(sess0928) - set(sess_pre)

In [107]:
f = open('/nfs/masi/MCL/file/txt/for_download/xnat20201002_mightallbad.txt', 'w')
for sess in need:
    f.write(sess + '\n')
f.close()